# Explore here

In [17]:
# Galerias
import pandas as pd
import numpy as np
import regex as re

import matplotlib.pyplot as plt
import seaborn as sns
import nltk
from nltk import download
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import os
from nltk.tokenize import word_tokenize
from sklearn.preprocessing import StandardScaler
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, mean_squared_error
from imblearn.over_sampling import SMOTE
from sklearn.svm import SVC
from sklearn.linear_model import LinearRegression
from pickle import dump
import streamlit as st
import warnings
from dotenv import load_dotenv
import os, joblib

load_dotenv()

api_key = os.getenv("API_KEY")
data_base = os.getenv("DATA_BASE_URL")

print(f"API_KEY: {api_key}, DATA_BASE: {data_base}")

API_KEY: Md2401, DATA_BASE: None


In [18]:
from flask import Flask
app = Flask(__name__)

@app.route("/")
def hello_world():
    return "Hello, World!"
print(app)

<Flask '__main__'>


In [19]:
# Descargo la data
df = pd.read_csv("../src/XAU_15m_data.csv", sep=";")
df

,Date,Open,High,Low,Close,Volume
0,2004.06.11 07:15,384.00,384.30,383.80,384.30,12
1,2004.06.11 07:30,383.80,384.30,383.60,383.80,12
2,2004.06.11 07:45,383.30,383.80,383.30,383.80,20
3,2004.06.11 08:00,383.80,384.10,383.60,383.60,8
4,2004.06.11 08:15,383.60,384.30,383.50,383.50,20
...,...,...,...,...,...,...
472585,2025.01.31 23:00,2801.15,2802.70,2799.64,2801.94,722
472586,2025.01.31 23:15,2801.89,2802.55,2800.62,2801.67,444
472587,2025.01.31 23:30,2801.64,2801.79,2798.50,2799.45,416
472588,2025.01.31 23:45,2799.62,2802.11,2799.23,2799.23,333


In [20]:
# Convertimo date a datatime
df['Date'] = pd.to_datetime(df['Date'])
df = df.sort_values('Date')
df

,Date,Open,High,Low,Close,Volume
0,2004-06-11 07:15:00,384.00,384.30,383.80,384.30,12
1,2004-06-11 07:30:00,383.80,384.30,383.60,383.80,12
2,2004-06-11 07:45:00,383.30,383.80,383.30,383.80,20
3,2004-06-11 08:00:00,383.80,384.10,383.60,383.60,8
4,2004-06-11 08:15:00,383.60,384.30,383.50,383.50,20
...,...,...,...,...,...,...
472585,2025-01-31 23:00:00,2801.15,2802.70,2799.64,2801.94,722
472586,2025-01-31 23:15:00,2801.89,2802.55,2800.62,2801.67,444
472587,2025-01-31 23:30:00,2801.64,2801.79,2798.50,2799.45,416
472588,2025-01-31 23:45:00,2799.62,2802.11,2799.23,2799.23,333


In [21]:
df.loc[:,'Next_Close'] = df['Close'].shift(-1)

# Eliminamos la última fila
df = df.dropna()
df

,Date,Open,High,Low,Close,Volume,Next_Close
0,2004-06-11 07:15:00,384.00,384.30,383.80,384.30,12,383.80
1,2004-06-11 07:30:00,383.80,384.30,383.60,383.80,12,383.80
2,2004-06-11 07:45:00,383.30,383.80,383.30,383.80,20,383.60
3,2004-06-11 08:00:00,383.80,384.10,383.60,383.60,8,383.50
4,2004-06-11 08:15:00,383.60,384.30,383.50,383.50,20,383.50
...,...,...,...,...,...,...,...
472584,2025-01-31 22:45:00,2798.79,2802.10,2798.58,2800.88,1468,2801.94
472585,2025-01-31 23:00:00,2801.15,2802.70,2799.64,2801.94,722,2801.67
472586,2025-01-31 23:15:00,2801.89,2802.55,2800.62,2801.67,444,2799.45
472587,2025-01-31 23:30:00,2801.64,2801.79,2798.50,2799.45,416,2799.23


In [22]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 472589 entries, 0 to 472588
Data columns (total 7 columns):
 #   Column      Non-Null Count   Dtype         
---  ------      --------------   -----         
 0   Date        472589 non-null  datetime64[ns]
 1   Open        472589 non-null  float64       
 2   High        472589 non-null  float64       
 3   Low         472589 non-null  float64       
 4   Close       472589 non-null  float64       
 5   Volume      472589 non-null  int64         
 6   Next_Close  472589 non-null  float64       
dtypes: datetime64[ns](1), float64(5), int64(1)
memory usage: 28.8 MB


In [23]:
df.describe()

,Date,Open,High,Low,Close,Volume,Next_Close
count,472589,472589.000000,472589.000000,472589.000000,472589.000000,472589.000000,472589.000000
mean,2014-12-07 03:29:35.033274112,1330.220561,1331.083877,1329.337321,1330.221168,860.813358,1330.226268
min,2004-06-11 07:15:00,381.300000,381.600000,381.100000,381.300000,1.000000,381.300000
25%,2009-10-20 17:45:00,1052.330000,1053.030000,1051.550000,1052.330000,176.000000,1052.330000
50%,2015-01-07 09:45:00,1291.090000,1291.700000,1290.470000,1291.090000,596.000000,1291.090000
75%,2020-01-20 11:30:00,1713.620000,1714.890000,1712.480000,1713.620000,1166.000000,1713.620000
max,2025-01-31 23:45:00,2815.450000,2817.230000,2812.260000,2815.640000,35966.000000,2815.640000
std,NaN,498.544097,498.829842,498.251434,498.549426,1046.262944,498.552077


In [24]:
df.head()
ultimo_valor = df.iloc[-1]
print(ultimo_valor[['Open', 'High', 'Low', 'Close', 'Volume']])

Open      2799.62
High      2802.11
Low       2799.23
Close     2799.23
Volume        333
Name: 472588, dtype: object


In [25]:
df = df[['Open', 'High', 'Low', 'Close', 'Volume', 'Next_Close']]  # Filtra columnas relevantes
df.info()
X = df[['Open', 'High', 'Low', 'Close', 'Volume']]
y = df['Next_Close']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)
print(X_train.shape, X_test.shape)

# Modelo lineal simple
model = LinearRegression()
model.fit(X_train, y_train)

y_pred = model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
print(f"Error cuadrático medio: {mse:.4f}")

<class 'pandas.core.frame.DataFrame'>
Index: 472589 entries, 0 to 472588
Data columns (total 6 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   Open        472589 non-null  float64
 1   High        472589 non-null  float64
 2   Low         472589 non-null  float64
 3   Close       472589 non-null  float64
 4   Volume      472589 non-null  int64  
 5   Next_Close  472589 non-null  float64
dtypes: float64(5), int64(1)
memory usage: 25.2 MB
(378071, 5) (94518, 5)
Error cuadrático medio: 3.4215


In [26]:
import joblib
model = joblib.load("../models/modelo_gold.pkl")
joblib.dump(model, 'modelo_gold.pkl')


['modelo_gold.pkl']

In [ ]:
# Cargar el modelo
model = joblib.load("modelo_gold.pkl")

# Título
st.title("Predicción del Precio del Oro")

# Inputs del usuario
open_price = st.number_input("Precio de Apertura (Open)", value=0.0)
high_price = st.number_input("Precio Máximo (High)", value=0.0)
low_price = st.number_input("Precio Mínimo (Low)", value=0.0)
close_price = st.number_input("Precio de Cierre (Close)", value=0.0)
volume = st.number_input("Volumen", value=0.0)

# Predicción
if st.button("Predecir Precio del Oro Futuro"):
    data = np.array([[open_price, high_price, low_price, close_price, volume]])
    prediction = model.predict(data)
    st.success(f"Predicción del Precio del Oro: ${prediction[0]:.2f}")



2025-05-19 17:31:39.478 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-19 17:31:39.480 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-19 17:31:39.482 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-19 17:31:39.482 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-19 17:31:39.483 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-19 17:31:39.484 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-19 17:31:39.484 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-19 17:31:39.485 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bar